In [34]:
from _keys import db_user, db_password, db_name, db_host, db_port
import psycopg2
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from nltk.tokenize import word_tokenize
stop=set(stopwords.words('english'))

In [35]:
sql = """
select *
	from (
		select author, lower(string_agg(title,',')) as corpus, 1 as is_bot
		from sus_user_posts
		where author in (select distinct author from sus_user_posts) and subreddit in (select subreddit from relevant_subreddit_info where is_relevant = 'yes')
		group by sus_user_posts.author
		having length(lower(string_agg(title,''))) >= 50)
		as posts_aggregate
union
select *
	from (
		select author, lower(string_agg(title,',')) as corpus, 0 as is_bot
		from norm_user_posts
		where author in (select distinct author from norm_user_posts limit 500) and subreddit in (select subreddit from relevant_subreddit_info where is_relevant = 'yes')
		group by norm_user_posts.author
		having length(lower(string_agg(title,''))) >= 50)
		as norm_agg
"""

corpus_sql = """
select string_agg(corpus, ',') as foo from (
select *
	from (
		select author, lower(string_agg(title,',')) as corpus, 1 as is_bot
		from sus_user_posts
		where author in (select distinct author from sus_user_posts) and subreddit in (select subreddit from relevant_subreddit_info where is_relevant = 'yes')
		group by sus_user_posts.author
		having length(lower(string_agg(title,''))) >= 50)
		as posts_aggregate
union
select *
	from (
		select author, lower(string_agg(title,',')) as corpus, 0 as is_bot
		from norm_user_posts
		where author in (select distinct author from norm_user_posts limit 500) and subreddit in (select subreddit from relevant_subreddit_info where is_relevant = 'yes')
		group by norm_user_posts.author
		having length(lower(string_agg(title,''))) >= 50)
		as norm_agg) as shit
"""

In [36]:
conn = psycopg2.connect(dbname=db_name, user=db_user, password=db_password, host=db_host, port=db_port)
posts_df = pd.read_sql(sql, conn)

In [37]:
cur = conn.cursor()
cur.execute(corpus_sql)
overall_corpus = cur.fetchall()[0][0]

In [38]:
def initialize_tfidf_df(master_corpus):
    data = {'author': ["DUMMY"], 'is_bot': [0.0], 'corpus': master_corpus}
    df = pd.DataFrame(data=data, index = [0])
    df = vectorizor(df.iloc[0])
    df = df.drop(df.index[0])
    return df

In [39]:
def vectorizor(row):
    try:
        corpus = row['corpus']
        author_name = row['author']
        is_bot_status = row['is_bot'].astype(int)
        cv = CountVectorizer(min_df=1)
        word_count_vector = cv.fit_transform([corpus])
        tfid_transformer = TfidfTransformer(smooth_idf=True, use_idf=True)
        tfid_transformer.fit(word_count_vector)
        tf_idf_vectors = tfid_transformer.transform(word_count_vector)
        feature_names = cv.get_feature_names_out()
        first_doc_vector = tf_idf_vectors[0]
        df = pd.DataFrame(first_doc_vector.T.todense(), index=feature_names, columns=['tfid'])
        df = df.transpose()
        df['author'] = author_name
        df['is_bot'] = is_bot_status
        return df
    except Exception as e:
        print(e)
        next

In [40]:
df = initialize_tfidf_df(overall_corpus)

In [41]:
for i in range(0,len(posts_df)):
    df = df.append(vectorizor(posts_df.iloc[i]))

In [42]:
df = df.fillna(0)

In [43]:
df.to_csv('testing_tfids.csv')

In [44]:
df.head()

,00,000,000th,02,039,04,05,06,07b,08,...,جردت,عليه,لازم,لغز,منتهى,والمهاره,पह,নত,蔡小煒,is_bot
tfid,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
tfid,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
tfid,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
tfid,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
tfid,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
